# Experimento B:

Goals:

1) Extender a classe de algum dos algoritmos no formato necessário para recomendações *user-user*.

2) Implementar o GridsSearch e validar seu uso para o nosso caso.

3) Adaptar uma varredura para diferentes quantidades de colunas acima do método do item 2.

O modelo receberá como entrar apenas o `id` da empresa e retornar uma lista do `N` mais recomendados (vizinhos mais próximos).

Verifiar: o modela terá entrada de empresas novas? Acho que não.

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [12]:
import numpy as np
import pandas as pd
from surprise import SVD, accuracy, Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [8]:
# Run this cell before close.
%watermark
%watermark --iversion
%watermark -b -r -g

2020-06-19T23:11:30+00:00

CPython 3.7.7
IPython 7.15.0

compiler   : GCC 8.3.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit
numpy  1.18.5
pandas 1.0.4

Git hash: b4348e2f24cd733e3f1939d40228356aa358edf2
Git repo: https://github.com/ysraell/aceleradev_private.git
Git branch: master


In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 1000)

# Carrega e processa o dataset:

In [10]:
path_data = '../data/'
top_cols = pd.read_csv('top_cols.csv')['cols'].to_list()
df_marked = pd.read_csv(path_data+'estaticos_market.csv', usecols=top_cols)
col_user = 'id'
top_cols.remove(col_user)

rest_cols = []
for col in top_cols:
    df_marked[col] = df_marked[col].fillna(0)*1

def normalize(x):
    return (x-np.min(x))/(np.max(x) - np.min(x)) if (np.max(x) - np.min(x)) > 0 else (x-np.min(x))

escala = 100
for col in top_cols:
    df_marked[col] = (escala*normalize(df_marked[col].tolist())).astype(np.uint8)
    
remove_cols = []
for col in top_cols:
    if df_marked[col].nunique() == 1:
        remove_cols.append(col)

df_marked = df_marked.drop(columns=remove_cols)
for col in remove_cols:
    top_cols.remove(col)

df_marked = pd.melt(df_marked, id_vars=["id"], var_name="itemID", value_name="rating").rename(columns={"id": "userID"})

reader = Reader(rating_scale=(0, escala))
data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']], reader)
del df_marked

# Modelo base.

In [13]:
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 18.2032


18.203169808345063

In [20]:
algo.trainset.all_users()

range(0, 462298)

In [22]:
algo.trainset.to_raw_uid(4)

'0293ec906c1746e1d0876f22c650bb8022f96c428300214dc973aa7a044bd30c'

In [23]:
algo.pu[4]

array([-0.33012803, -0.87753922, -0.16000786,  0.33690633, -0.44181853,
        0.31283246, -0.40416128,  0.673524  , -0.74984166,  0.12221613,
       -0.7911641 ,  0.14049286, -0.37121182,  0.89099278, -0.36245022,
       -0.29306878,  0.29103981,  0.525298  , -0.45211667,  0.22572351,
        0.17184372,  0.58922408,  0.41421369,  0.28501298, -0.19561715,
       -0.27319747, -1.44398452, -0.84475615, -1.23579915, -0.10919373,
       -0.52979   , -0.12286462, -1.41653664, -0.61992774,  0.28968281,
        0.67964255, -0.52213828,  0.25314298,  0.31413346, -0.42081662,
       -0.68342602,  0.43810118, -0.32876472,  0.86476891, -0.41420993,
       -0.46208325, -0.6581106 , -0.07915893,  0.23415733,  0.86767033,
       -0.14446805, -0.28450242,  0.99261077, -0.98603909,  0.64242801,
        0.17820183,  0.02043893,  0.36393089,  0.03456342,  0.28366136,
       -0.70158252,  0.54591876,  0.27118546,  0.15735987,  0.23547393,
        0.10194708,  0.8354555 ,  0.61943589,  0.88713212, -0.01